## **CREATING THE MODEL**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 25
batch_size = 128
learning_rate = 0.001

In [ ]:
# Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Normalization for validation/test
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [ ]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=train_transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True)

In [ ]:
# Function to calculate accuracy
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)


In [ ]:
# Training the model
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_acc += accuracy(outputs, labels) * inputs.size(0)
        # Average loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_acc += accuracy(outputs, labels) * inputs.size(0)

    # Average validation loss and accuracy
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_acc / len(val_loader.dataset)

    # Adjust learning rate based on validation accuracy
    scheduler.step(val_acc)

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

print('Finished Training')

Epoch [1/25], Train Loss: 1.5475, Train Acc: 0.4376, Val Loss: 1.2202, Val Acc: 0.5611
Epoch [2/25], Train Loss: 1.2087, Train Acc: 0.5678, Val Loss: 0.9970, Val Acc: 0.6410
Epoch [3/25], Train Loss: 1.0554, Train Acc: 0.6216, Val Loss: 0.9143, Val Acc: 0.6798
Epoch [4/25], Train Loss: 0.9645, Train Acc: 0.6591, Val Loss: 0.8967, Val Acc: 0.6899
Epoch [5/25], Train Loss: 0.8933, Train Acc: 0.6858, Val Loss: 0.8062, Val Acc: 0.7190
Epoch [6/25], Train Loss: 0.8371, Train Acc: 0.7065, Val Loss: 0.7847, Val Acc: 0.7218
Epoch [7/25], Train Loss: 0.7985, Train Acc: 0.7186, Val Loss: 0.7585, Val Acc: 0.7379
Epoch [8/25], Train Loss: 0.7628, Train Acc: 0.7332, Val Loss: 0.7342, Val Acc: 0.7461
Epoch [9/25], Train Loss: 0.7313, Train Acc: 0.7438, Val Loss: 0.7374, Val Acc: 0.7437
Epoch [10/25], Train Loss: 0.7023, Train Acc: 0.7533, Val Loss: 0.7009, Val Acc: 0.7615
Epoch [11/25], Train Loss: 0.6846, Train Acc: 0.7618, Val Loss: 0.6814, Val Acc: 0.7630
Epoch [12/25], Train Loss: 0.6561, Train 

## **TESTING THE MODEL ON A SAMPLE IMAGE**

In [ ]:
# Load the image
from PIL import Image
img_path = "/content/image.jpg"
image = Image.open(img_path)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # Example normalization
])

# Apply the transformation
normalized_tensor_image = transform(image)

# Verify the size of the normalized tensor
print(normalized_tensor_image.size())  # Should print torch.Size([3, 32, 32])


torch.Size([3, 32, 32])


In [ ]:
model.eval()
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
with torch.no_grad():
    op = model(normalized_tensor_image)
    _, predicted = torch.max(op, 1)

# Print the predicted class
print(f'Predicted class: {classes[predicted.item()]}')

Predicted class: cat


In [ ]:
# Classify the image and get probabilities
with torch.no_grad():
    outputs = model(normalized_tensor_image)
    probabilities = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities

# Print the probabilities for each class
probabilities = probabilities.cpu().numpy().flatten()  # Move to CPU and flatten the tensor
for i, prob in enumerate(probabilities):
    print(f' {classes[i]}: {prob:.4f}')

 plane: 0.0009
 car: 0.0026
 bird: 0.0003
 cat: 0.8720
 deer: 0.0001
 dog: 0.0183
 frog: 0.0001
 horse: 0.0750
 ship: 0.0016
 truck: 0.0292


## **app.py**

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# Ensure you have the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the trained model weights
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('cifar10_model.pth', map_location=device))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def predict(image):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return classes[predicted.item()]

# Streamlit app
st.title("Image Classification with CIFAR-10")
st.write("Upload an image to classify")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    label = predict(image)
    st.write(f"The uploaded image is classified as: {label}")


Overwriting app.py


## **WEB APP RELATED**

In [ ]:
# save
torch.save(model.state_dict(), 'cifar10_model.pth')

In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.231.220.25


In [ ]:
!streamlit run app.py &>/dev/null& npx localtunnel --port 8501

npx: installed 22 in 3.618s
your url is: https://sad-bottles-shine.loca.lt
^C
